In [1]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import os
import datetime
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

import pickle
import sys
import matplotlib.pyplot as plt
# just set the seed for the random number generator
np.random.seed(107)

import numpy as np
np.random.seed(1) # NumPy
import random
random.seed(3) # Python
import tensorflow as tf

from keras import backend as K

import matplotlib.cm as cm

from sklearn.manifold import TSNE

from keras.layers import Dense, Flatten, LSTM, Dropout

In [2]:
# Usando Google Colab:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/Python/MEU_TCC/Classes')
pasta = str("/content/drive/MyDrive/Python/MEU_TCC/Data/")
print(pasta)

preços = pd.read_csv(pasta + "Preços.csv",index_col='Data',parse_dates=True)
semestres = pd.read_csv(pasta + "Semestres.csv",parse_dates=True)

Datas = pd.DataFrame(preços.index)
Datas['indice'] = Datas.index
log_data = np.log(preços)


Mounted at /content/drive
/content/drive/MyDrive/Python/MEU_TCC/Data/


In [3]:
%load_ext autoreload
%aimport class_SeriesAnalyser_Distance, class_Trader_Distance, class_DataProcessor
%autoreload 1

series_analyser = class_SeriesAnalyser_Distance.SeriesAnalyser()
trader = class_Trader_Distance.Trader()
data_processor = class_DataProcessor.DataProcessor()

In [4]:
top_n = 5
subsample = 2500
min_half_life = 0 # number of points in a day
max_half_life = 200000 #~number of points in a year: 78*252
file_extension = 'intraday'

# Calculando Renda fixa anualizada:

In [5]:
pastaC = str("/content/drive/MyDrive/Python/MEU_TCC/Data/")
CDI_ret = pd.read_csv(pastaC + "CDI_ret.csv", )
# Convert the 'data' column to datetime format
CDI_ret.index = pd.to_datetime(CDI_ret['Data'])
CDI_ret.drop(['Data'],inplace=True,axis=1)

CDI_ret = CDI_ret[1:]

In [6]:
anoi = 1996
rfannualized = pd.DataFrame(columns=['retorno_anualizado'])
inicio = 0

for nsem in range(0, len(semestres) - 2,2):
    print(f"Starting period {nsem} | Past days: {inicio}")
    ano = math.floor(anoi + nsem/2)
    print(ano)

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][nsem]).query("Data == True").index[0]
    inicio = Datas.iloc[inicio][0]

    twelve_months = (Datas == semestres['Data'][nsem + 2]).query("Data == True").index[0]
    twelve_months = Datas.iloc[twelve_months][0]
    print('inicio: ',inicio,' twelve_months: ',twelve_months)


    annualized_ret = (np.cumprod(1 + CDI_ret['CDI Acumulado'][inicio:twelve_months]) - 1)[-1]
    rfannualized.loc[ano] = annualized_ret

rf_mean_ano = rfannualized.mean()[0]

Starting period 0 | Past days: 0
1996
inicio:  1996-01-02 00:00:00  twelve_months:  1996-12-31 00:00:00
Starting period 2 | Past days: 1996-01-02 00:00:00
1997
inicio:  1996-12-31 00:00:00  twelve_months:  1997-12-31 00:00:00
Starting period 4 | Past days: 1996-12-31 00:00:00
1998
inicio:  1997-12-31 00:00:00  twelve_months:  1998-12-31 00:00:00
Starting period 6 | Past days: 1997-12-31 00:00:00
1999
inicio:  1998-12-31 00:00:00  twelve_months:  1999-12-31 00:00:00
Starting period 8 | Past days: 1998-12-31 00:00:00
2000
inicio:  1999-12-31 00:00:00  twelve_months:  2000-12-29 00:00:00
Starting period 10 | Past days: 1999-12-31 00:00:00
2001
inicio:  2000-12-29 00:00:00  twelve_months:  2001-12-31 00:00:00
Starting period 12 | Past days: 2000-12-29 00:00:00
2002
inicio:  2001-12-31 00:00:00  twelve_months:  2002-12-31 00:00:00
Starting period 14 | Past days: 2001-12-31 00:00:00
2003
inicio:  2002-12-31 00:00:00  twelve_months:  2003-12-31 00:00:00
Starting period 16 | Past days: 2002-12

#No Cluster - Distância

In [7]:
inicio = 0
pairs_t = []
performance_threshold_test_t = []
sharpe_results_threshold_test_nocosts_t = []
cum_returns_threshold_test_nocosts_t = []
results_t = pd.DataFrame(columns=['n_pairs','Daily_sharpe_ratio','avg_annual_roi',
                                  'Total Number of Trades','Positive Trades','Negative Trades','pct_positive_trades_per_pair',
                                  'pct_pairs_with_positive_returns','avg_half_life','avg_hurst_exponent','Maximum Drawdown'])

Pares_Total = pd.DataFrame(columns=['Ticker 1','Ticker 2','Erro Quadrático Médio'])

for big_loop in range(0, len(semestres) - 3):
    print(f"\nStarting period {big_loop} | Past days: {inicio}")

    # Listando os dias dos intervalos
    inicio = (Datas == semestres['Data'][big_loop]).query("Data == True").index[0]
    inicio = Datas.iloc[inicio][0]
    twelve_month = (Datas == semestres['Data'][big_loop + 2]).query("Data == True").index[0]
    twelve_months = Datas.iloc[twelve_month][0]
    inicio_test = Datas.iloc[twelve_month+1][0]
    six_months = (Datas == semestres['Data'][big_loop + 3]).query("Data == True").index[0]
    six_months = Datas.iloc[six_months][0]

    # split data in training and test
    df_prices_train, df_prices_test = data_processor.split_data(preços,(inicio, twelve_months),(inicio_test,six_months),remove_nan=True)

    # interday
    n_years_test = round(len(df_prices_train)/(250))

    no_cluster_series = pd.Series(0,index = df_prices_train.columns)
    pairs_all_a_all, unique_tickers = series_analyser.get_candidate_pairs(clustered_series=no_cluster_series,
                                                                          pricing_df_train=df_prices_train,
                                                                          pricing_df_test=df_prices_test,
                                                                          min_half_life=min_half_life,
                                                                          max_half_life=max_half_life,
                                                                          min_zero_crosings=12,
                                                                          hurst_threshold= 1,
                                                                          subsample=0,
                                                                          top_n=top_n
                                                                          )
    print('Out of {} possible combinations'.format(len(df_prices_train.columns)*(len(df_prices_train.columns)-1)/2))

    results_without_costs, results_with_costs, performance_threshold_test = \
            trader.apply_trading_strategy(pairs_all_a_all,
                                          strategy='fixed_beta',
                                          entry_multiplier=2,
                                          exit_multiplier=0,
                                          test_mode=True,
                                          train_val_split=0,
                                          rf = rfannualized
                                          )

    sharpe_results_threshold_test_nocosts, cum_returns_threshold_test_nocosts = results_without_costs
    #sharpe_results_threshold_test_w_costs, cum_returns_threshold_test_w_costs = results_with_costs

    # interday
    n_years_test = round(len(df_prices_test)/(240))

    n_years_test

    results, pairs_summary = trader.summarize_results(sharpe_results_threshold_test_nocosts,
                                                      cum_returns_threshold_test_nocosts,
                                                      performance_threshold_test,
                                                      pairs_all_a_all,
                                                      n_years_test,
                                                      rf = rfannualized)

    for i in range(0, len(pairs_all_a_all)):
      Pares_Total = Pares_Total.append({'Ticker 1': pairs_all_a_all[i][0], 'Ticker 2': pairs_all_a_all[i][1], 'Erro Quadrático Médio': pairs_all_a_all[i][2]['SSE']},ignore_index=True)

    results_t = results_t.append(results,ignore_index=True)

results_t['Annualized SR'] = results_t['Daily_sharpe_ratio']*(252**0.5)

Pares_Total['Ticker Combinado'] = Pares_Total[['Ticker 1', 'Ticker 2']].apply(lambda x: '-'.join(sorted(x)), axis=1)
Pares_Total = Pares_Total.drop_duplicates(subset=['Ticker Combinado'], keep='first')

Output hidden; open in https://colab.research.google.com to view.

In [8]:
pd.set_option('display.float_format', '{:.10e}'.format)


In [9]:
pastaP = str("/content/drive/MyDrive/Python/MEU_TCC/Data/pickle/")

with open(pastaP + 'Pares_Distancia_Fixed_Beta_5.pickle', 'wb') as f:
    pickle.dump(results_t, f)

with open(pastaP + 'Pares_Distancia_Lista_5.pickle', 'wb') as f:
    pickle.dump(Pares_Total, f)

In [10]:
pastaP = str("/content/drive/MyDrive/Python/MEU_TCC/Data/pickle/")

with open(pastaP + 'Pares_Distancia_Fixed_Beta_5.pickle', 'rb') as handle:
    results_t = pickle.load(handle)

with open(pastaP + 'Pares_Distancia_Lista_5.pickle', 'rb') as handle:
    Pares_Total = pickle.load(handle)

In [11]:
#Sharpe Ratio Anualizado:
SRA = (results_t['avg_annual_roi'].mean())/np.std(results_t['avg_annual_roi'])

In [12]:
# Resultados:
print('Top N Pares:',results_t['n_pairs'].min())
#print('Sharpe Ratio Anualizado: {:.3f}'.format(results_t['Annualized SR'].mean()))
print('Sharpe Ratio Anualizado: {:.3f}'.format(SRA))
print('Excesso de Retornos Anualizado: {:.3f} %'.format(results_t['avg_annual_roi'].mean()))
print('MDD: {:.3f} %'.format(results_t['Maximum Drawdown'].min()))
print('Total de Pares:', Pares_Total['Ticker Combinado'].count()) # Não está contando a repetição dos pares
print('Pares rentáveis: {:.3f} %'.format(results_t['pct_pairs_with_positive_returns'].mean()))
print('Total de Transações:', results_t['Total Number of Trades'].sum())
print('Operações rentáveis:', results_t['Positive Trades'].sum())
print('Operações deficitárias :', results_t['Negative Trades'].sum())

Top N Pares: 5
Sharpe Ratio Anualizado: 0.237
Excesso de Retornos Anualizado: 3.888 %
MDD: -48.471 %
Total de Pares: 151
Pares rentáveis: 50.000 %
Total de Transações: 306
Operações rentáveis: 160
Operações deficitárias : 146


In [13]:
rf_mean_ano*100

14.244056961092486

In [14]:
results_t['avg_annual_roi'] + rf_mean_ano*100

0     3.0010022102e+01
1     4.2867360780e+01
2     8.1181594945e+00
3     1.4200134274e+01
4     6.8249640015e+01
5     2.1782221279e+01
6     5.0008105465e+01
7     1.6728925220e+01
8     1.9167658312e+01
9    -2.2803073730e+01
10   -1.3695226823e+01
11    6.2021944862e+01
12   -4.5542999601e+00
13    8.3586813888e+00
14    1.5498288310e+01
15    4.8510153632e+00
16    8.4774236407e+00
17    2.7451594921e+01
18    2.2245802372e+01
19    8.2539615345e+00
20    2.0101217233e+01
21    1.3122183006e+01
22    9.3435595287e+00
23    2.0384938688e+01
24    7.3080530499e+00
25    1.6442992508e+01
26    1.3649392508e+01
27    2.1982099623e+01
28    2.8035184497e+00
29    3.3345311190e+01
30    2.3863142101e+01
31    1.6059604284e+01
32    1.5288523578e+01
33    8.0367912229e+00
34    2.3086595239e+01
35    2.2223934849e+01
36    1.5376513876e+01
37    2.2651344060e+01
38    5.7569568324e+00
39   -8.7538480945e+00
40    1.8326990463e+01
41    2.9228991473e+01
42    8.5530968950e+00
43    5.025

In [15]:
rfsem = []
rfsem[rfannualized.iloc[0][0]]
for s in range(1,len(rfannualized)):
  print(rfannualized.iloc[s])
  rfsem = rfsem.append(rfannualized.iloc[s][0],ignore_index=True)

TypeError: list indices must be integers or slices, not numpy.float64

In [ ]:
rfsem[rfannualized.iloc[0][0]]
rfsem

In [ ]:
results_t['Annualized SR'].mean()

In [ ]:
results_t